In [1]:
import pandas as pd
import dotenv
import os
from libraries.client_stashapp import get_stashapp_client
from libraries.StashDbClient import StashDbClient

dotenv.load_dotenv()

stash = get_stashapp_client()


dUsing stash (v0.27.2-21-g4ab6d773) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...


In [16]:
scenes_with_dupes = stash.find_scenes({ 
  "file_count": {
    "modifier": "GREATER_THAN",
    "value": 1
  }
}, fragment="id title date studio { name } files { path size fingerprints { type value } }")

# Create a dictionary to group files by oshash
oshash_groups = {}

# Go through each scene and its files
for scene in scenes_with_dupes:
    # Get all oshash values for this scene's files
    for file in scene['files']:
        oshash = next((fp['value'] for fp in file['fingerprints'] if fp['type'] == 'oshash'), None)
        if oshash:
            if oshash not in oshash_groups:
                oshash_groups[oshash] = []
            oshash_groups[oshash].append({
                'scene_id': scene['id'],
                'title': scene['title'],
                'path': file['path']
            })

# Print files with matching oshashes
for oshash, files in oshash_groups.items():
    if len(files) > 1:
        for file in files:
            title = file['title'][:40] + '...' if len(file['title']) > 40 else file['title']
            print(f"\"{file['path']}\"")
        print()